In [35]:
import math
import pandas as pd
from enum import Enum

class MessageKind(Enum):
    MEASURE = "MEASURE"
    GENREQ = "GENREQ"
    GENRESP = "GENRESP"
    
class ReqResKind(Enum):
    READ = "READ"
    COMMAND = "COMMAND"
    UNSET = "UNSET"
    
class AtkStatus(Enum):
    UNMOD = "UNMOD"
    COMPR = "COMPR"
    BLOCK = "BLOCK"
    
class EvilState(Enum):
    INACTIVE = "INACTIVE"
    MIDDLE = "MIDDLE"
    FULL = "FULL"
    
class Keys(Enum):
    ID = "id"
    MESSAGEKIND = "messageKind"
    REQRESKIND = "reqResKind"
    ATKSTATUS = "atkStatus"
    DATA = "data"
    EVILSTATE = "evilState"
    CREATIONTIME = "creationTime"
    TIMESTAMP = "timestamp"

class OutKeys(Enum):
    MITM = "MITM"
    SRM = "SRM"
    UC = "UC"
    UPS = "UPS"
    IMD = "IMD"
    MC = "MC"
    CC = "CC"
    
    
evilStateToVar = {
    EvilState.INACTIVE.value: 1,
    EvilState.MIDDLE.value: 2,
    EvilState.FULL.value: 3
}
    
# Simulation time limit in seconds
simTimeLimit = 120
# Number of runs
numRuns = 1
# Connection index (0 for server[0] etc...)
connIndex = 0
# Discretization time in seconds
deltaT = 5


relPath = "../logs/"
evilFileName = "evilClient.app[" + connIndex.__str__() + "].csv"
cliFileName = "client[" + connIndex.__str__() + "].app[0].csv"
serFileName = "server[" + connIndex.__str__() + "].app[0].csv"


In [36]:
for i in range(numRuns):
    complRelPath = relPath + "run" + i.__str__() + "/"
    evilDf = pd.read_csv(complRelPath + evilFileName)
    cliDf = pd.read_csv(complRelPath + cliFileName)
    evilDf = evilDf.drop(Keys.CREATIONTIME.value, axis=1)
    
    currT = 0
    MITMv = 0
    SRMv = 0
    UCv = 0
    MCv = 0
    CCv = 0
    outDf = pd.DataFrame(columns=[OutKeys.MITM.value, OutKeys.SRM.value, OutKeys.UC.value, OutKeys.MC.value, OutKeys.CC.value])
    while currT < simTimeLimit:
        evilWindowDf = evilDf[evilDf[Keys.TIMESTAMP.value].between(currT, currT + deltaT, inclusive="left")]
        cliWindowDf = cliDf[cliDf[Keys.TIMESTAMP.value].between(currT, currT + deltaT, inclusive="left")]
        evilTotNumWindow = evilWindowDf.shape[0]
        cliTotNumWindow = cliWindowDf.shape[0]
        if not evilWindowDf.empty:
            MITMv = evilStateToVar[evilWindowDf.iloc[-1][Keys.EVILSTATE.value]]
            numComprMeas = evilWindowDf[((evilWindowDf[Keys.MESSAGEKIND.value] == MessageKind.MEASURE.value) | 
                                            (evilWindowDf[Keys.REQRESKIND.value] == ReqResKind.READ.value)) & 
                                            (evilWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            numComprCmds = evilWindowDf[(evilWindowDf[Keys.REQRESKIND.value] == ReqResKind.COMMAND.value) & 
                                            (evilWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            SRMv = max(math.trunc((numComprMeas/evilTotNumWindow) * 4), SRMv)
            UCv = max(math.trunc((numComprCmds/evilTotNumWindow) * 4), UCv)
        if not cliWindowDf.empty:
            numComprMeas = cliWindowDf[((cliWindowDf[Keys.MESSAGEKIND.value] == MessageKind.MEASURE.value) | 
                                            (cliWindowDf[Keys.REQRESKIND.value] == ReqResKind.READ.value)) & 
                                            (cliWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            numComprCmds = cliWindowDf[(cliWindowDf[Keys.REQRESKIND.value] == ReqResKind.COMMAND.value) & 
                                            (cliWindowDf[Keys.ATKSTATUS.value] == AtkStatus.COMPR.value)].shape[0]
            MCv = max(math.trunc((numComprMeas/cliTotNumWindow) * 4), MCv)
            CCv = max(math.trunc((numComprCmds/cliTotNumWindow) * 4), CCv)
        outDf.loc[len(outDf.index)] = [MITMv, SRMv, UCv, MCv, CCv]

        currT += deltaT
    print(outDf)
    outDf.to_csv("outTrain.csv", index=False)


    MITM  SRM  UC  MC  CC
0      0    0   0   0   0
1      1    0   0   0   0
2      1    0   0   0   0
3      1    0   0   0   0
4      2    0   0   0   0
5      2    1   0   1   0
6      2    1   1   1   0
7      2    1   1   1   0
8      3    1   1   1   0
9      3    1   1   1   0
10     3    1   1   1   0
11     3    1   1   1   0
12     3    1   1   1   0
13     3    1   1   1   1
14     3    2   1   1   1
15     3    2   1   1   1
16     3    2   1   1   1
17     3    2   1   1   1
18     3    2   1   1   1
19     3    2   1   1   1
20     3    2   1   1   1
21     3    2   1   1   1
22     3    2   1   1   1
23     3    2   1   1   1
